In [82]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from copy import deepcopy
import gensim
import pymorphy2
import re
import os

In [ ]:
for i in os.listdir('data_test'):
    with open('data_test/Соцсети1_новость.txt') as t:
        text = t.readlines()
        for i in range(len(text)):
            text[i] = text[i].strip('\n')
        text = '.'.join(text)
        text = re.sub(r'\.+', ".", text)

In [242]:
# pretrained_ft = gensim.models.fasttext.load_facebook_model('ru.bin', encoding='utf-8')

In [243]:
def preprocessing_rus(sentence):
    sentence = sentence.lower()
    sentence = re.sub('[^А-Яа-яA-Za-z]+', ' ', sentence).strip()
    return sentence

def mean_sent_vector(sentence_tokens):
    return np.array([np.mean([pretrained_ft[w] for w in sentence_tokens if w in pretrained_ft]
                     or [np.zeros(300)], axis=0)])


sents = [sent.split('.') for sent in sent_tokenize(text)]
sents = [item for sublist in sents for item in sublist if len(item) >= 30
                                                        and '?' not in item]
sents_storage = deepcopy(sents)
n_sents = int(round(np.sqrt(len(sents_storage))))

for i in range(len(sents)):
    sents[i] = preprocessing_rus(sents[i])
    sents[i] = word_tokenize(sents[i])
    for j in range(len(sents[i])):
        sents[i][j] = pymorphy2.MorphAnalyzer().parse(sents[i][j])[0].normal_form

In [244]:
sent_vectors = []
for sent in sents:
    sent_vectors.append(np.squeeze(mean_sent_vector(sent)))
sent_vectors = np.asarray(sent_vectors)

/Users/user007/Documents/Workspace/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/Users/user007/Documents/Workspace/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [245]:
kmeans = KMeans(n_clusters=n_sents).fit(sent_vectors)

In [246]:
# pca = PCA(n_components=2, random_state=0)
# reduced_features = pca.fit_transform(sent_vectors)

# # reduce the cluster centers to 2D
# reduced_cluster_centers = pca.transform(kmeans.cluster_centers_)

In [247]:
# plt.figure(figsize=(15,5))
# plt.scatter(reduced_features[:,0], reduced_features[:,1], c=kmeans.predict(sent_vectors))
# plt.scatter(reduced_cluster_centers[:, 0], reduced_cluster_centers[:,1], marker='x', s=150, c='b')

In [248]:
centre_sents_idx = []
for j in range(n_sents):
    idx_cand = np.where(kmeans.labels_ == j)
    vec_cand = sent_vectors[idx_cand]
    closest, dist = pairwise_distances_argmin_min([kmeans.cluster_centers_[j]], vec_cand)
    centre_sents_idx.append(idx_cand[0][int(closest)])
centre_sents_idx = sorted(centre_sents_idx)
summary = '. '.join([sents_storage[i] for i in centre_sents_idx])


In [249]:
print(summary)

Facebook стала отключать счётчики лайков пользователям в Австралии . Сейчас Facebook собирает отзывы пользователей, от них будет зависеть запуск функции в других странах, пояснил представитель соцсети. С апреля 2019 года компания тестирует функцию скрытия количества лайков под публикациями в Instagram, в зону тестирования включены пользователи семи стран
